In [24]:
import torch
import torchvision
import torchvision.transforms as transforms

from datasets import CustomCIFAR10 as CIFAR10_dataset
from victim_model import *
from utils import *

from consts import *

from torch.utils.tensorboard import SummaryWriter
import pickle

In [25]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(cifar10_mean, cifar10_std)
    ])
batch_size = 32

trainset= CIFAR10_dataset("../data/", transform=transform, train = True)
testset= CIFAR10_dataset("../data/", transform=transform, train = False)

trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=1, shuffle=False)

In [26]:
torch.hub.set_dir('/mnt/DONNEES/hbrachemi/.cache/torch/hub/checkpoints/')

In [27]:
model_ft= VictimModel("vgg16",True,10)

In [28]:
hyperparametters = {}
hyperparametters["sigma"]=0.1
hyperparametters["lambda"]=1
hyperparametters["criterion"] = torch.nn.CrossEntropyLoss()
hyperparametters["sponge_optimizer"] = torch.optim.SGD(model_ft.model.parameters(),lr=0.01, momentum=0.9,weight_decay= 5e-4)
hyperparametters["num_sponge_epochs"] = 50
hyperparametters["sponge_criterion"] = "l0"
hyperparametters["num_epochs"] = 10
hyperparametters["criterion"] = torch.nn.CrossEntropyLoss()
hyperparametters["optimizer"] = torch.optim.Adam(model_ft.model.parameters(),lr=0.1)


In [30]:
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [29]:
writer = SummaryWriter()

In [ ]:
#CLEAN TRAINING

#a = model_ft.train({"train":trainloader,"val":testloader},hyperparametters,writer=writer)
#PATH = "../weights_sponge_backdoor/clean/vgg16.pt"

PATH ='../weights_sponge_backdoor/sponge/vgg16_non_scratch.pt'
model_ft.model.load_state_dict(torch.load(PATH))

a=model_ft.evaluate(testloader)

In [ ]:
#SPONGE TRAINING
#p_ids = [random.randint(1,len(trainset)) for _ in range(int(0.05*len(trainset)))]
pickle_in = open("p_ids_0.05.pickle",'rb')
p_ids = pickle.load(pickle_in)


dataloaders= {"train":trainloader,"val":testloader}

a = model_ft.sponge_train(dataloaders,
                          p_ids,
                          hyperparametters,
                          writer)

model_ft.evaluate(testloader)